In [27]:

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

#warnings.filterwarnings(action='once')
import pandas as pd
from pandas import MultiIndex, Int16Dtype

import numpy as np
import math

from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, f1_score
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV, PredefinedSplit
# models
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.svm import LinearSVR
import xgboost as xgb
from sklearn.ensemble import ExtraTreesRegressor

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

from sklearn import ensemble
from sklearn import datasets
from sklearn import model_selection

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

In [28]:
#!pip3 install xgboost --upgrade

In [29]:
#df_train = pd.read_csv('../data/train_encoded_pca_cum.csv')
#df_test = pd.read_csv('../data/test_encoded_pca_cum.csv')
#df_train = pd.read_csv('../data/train_encoded_pca.csv')
#df_test = pd.read_csv('../data/test_encoded_pca.csv')

df_train = pd.read_csv('../data/train_encoded_1.csv')
df_test = pd.read_csv('../data/test_encoded_1.csv')
df_test = df_test.reindex(columns=df_train.columns.intersection(df_test.columns))

#train_encoded_pca_cum
#train_encoded_pca
#ex = pd.read_csv('../data/example-submission.csv')

In [37]:
df_train.columns
df_train.drop(columns = [

Index(['num_beds', 'num_baths', 'size_sqft', 'lat', 'lng', 'price',
       'built_year', 'pri_sch', 'sec_sch', 'mrt_station', 'population_density',
       'closest_dist_to_mrt', 'closest_dist_to_pri', 'closest_dist_to_sec',
       'close_pri_sch', 'close_sec_sch', 'closest_dist_to_shop',
       'closest_dist_to_com', 'price_per_sqft', 'price_avg_planning_area',
       'psf_bbox', 'age', 'property_type_cluster house', 'property_type_condo',
       'property_type_corner terrace', 'property_type_executive condo',
       'property_type_hdb', 'property_type_semi-detached house',
       'property_type_terraced house', 'property_type_infrequent_sklearn',
       'furnishing_partial', 'furnishing_unfurnished',
       'furnishing_unspecified', 'furnishing_infrequent_sklearn',
       'tenure_999-year leasehold', 'tenure_freehold'],
      dtype='object')

In [30]:
from sklearn.model_selection import train_test_split
np.random.seed(0)

train, valid = train_test_split(df_train, test_size=0.2)

In [31]:
X_train = train.drop(['price', 'price_per_sqft'], axis=1)
y_train = train['price_per_sqft']

X_valid = valid.drop(['price', 'price_per_sqft'], axis=1)
y_valid = valid['price_per_sqft']

df_test_size_sqft = df_test['size_sqft'].copy()
#df_test= df_test.drop(['size_sqft'], axis=1)

idxs = np.arange(X_train.shape[0])
np.random.shuffle(idxs)
df_test.columns.difference(X_train.columns)


Index([], dtype='object')

In [32]:
X_train.columns.difference(df_test.columns)


Index([], dtype='object')

In [33]:
warnings.filterwarnings('ignore')

model = ExtraTreesRegressor()

parameters = {'n_estimators':[2500], #when use hyperthread, xgboost may become slower
              'max_depth': [3],
              'min_samples_split':[2],
              'min_samples_leaf':[1],
              'max_features':['auto'],
              'bootstrap': [True],
              'n_jobs':[-1],
              'warm_start':[True],
              'random_state' :[42]}

# #parameters = {'n_estimators':[2500, 3000], #when use hyperthread, xgboost may become slower
#               'max_depth': [2,3],
#               'min_samples_split':[2, 5, 10],
#               'min_samples_leaf':[1,5,10],
#               'max_features':['auto', 'sqrt', 'log2', 1],
#               'bootstrap': [True],
#               'n_jobs':[-1],
#               'warm_start':[True, False],
#               'random_state' :[42]}

clf = GridSearchCV(model, parameters, n_jobs=5, 
                   cv=KFold(n_splits=2, shuffle=True, random_state=42), 
                   scoring='neg_root_mean_squared_error',
                refit=True)

clf.fit(X_train, y_train)

yt_pred = clf.predict(X_train)
yv_pred = clf.predict(X_valid)

print('Training RMSE: {:.3f}'.format(mean_squared_error(yt_pred, y_train, squared=False)))
print('Test RMSE: {:.3f}'.format(mean_squared_error(yv_pred, y_valid, squared=False)))


/usr/local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Training RMSE: 429.475
Test RMSE: 424.566


In [35]:
print(clf.best_params_)

print(clf.best_estimator_)

print(clf.best_score_)

print(clf.predict(df_test.copy()))


{'bootstrap': True, 'max_depth': 3, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 2500, 'n_jobs': -1, 'random_state': 42, 'warm_start': True}
ExtraTreesRegressor(bootstrap=True, max_depth=3, n_estimators=2500, n_jobs=-1,
                    random_state=42, warm_start=True)
-425.6610757090351
[2281.20062518 1775.17687017 2662.48557638 ... 2834.61936377  670.19342566
 2554.58446438]


In [36]:
yTest = clf.predict(df_test)

yTest = yTest * df_test_size_sqft
yTest = pd.DataFrame(yTest.values, columns=['Predicted'], index=df_test.index)

yTest.to_csv('../predictions/extra_trees_1.csv')

In [ ]:
#scaler = preprocessing.MinMaxScaler().fit(X_train)

# xgb_model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42, eval_metric=["error"])
# xgb_model.fit(X_train, y_train)

# # Predict labels for test data
# yt_pred = xgb_model.predict(X_train)
# yv_pred = xgb_model.predict(X_valid)

# print('Training RMSE: {:.3f}'.format(mean_squared_error(yt_pred, y_train, squared=False)))
# print('Test RMSE: {:.3f}'.format(mean_squared_error(yv_pred, y_valid, squared=False)))

# print(f'Percentage error for training data {((abs(yt_pred - y_train)/y_train)*100).mean()}')
# print(f'Percentage error for validation data {((abs(yv_pred - y_valid)/y_valid)*100).mean()}')

# xgb.plot_importance(xgb_model, height = 5, max_num_features=20)

# score = cross_val_score(xgb_model, X_train, y_train, n_jobs=-1, cv=5, scoring="r2")
# r2_mean = score.mean()

# print(f"R2 mean: {r2_mean}")
#yTest = xgb_model.predict(df_test)

# yTest = yTest * df_test_size_sqft
# yTest = pd.DataFrame(yTest.values, columns=['Predicted'], index=df_test.index)

# yTest.to_csv('../predictions/xgboost.csv')